In [1]:
import cffi
import numpy as np
import ufl
import scipy as sp
from dolfinx import fem, jit, mesh
from dolfinx.fem import petsc
from mpi4py import MPI
from petsc4py import PETSc

In [2]:
delta = 1.0
gamma = 0.5
nx, ny = 100, 100
cell_index = 0

In [3]:
domain = mesh.create_rectangle(
    MPI.COMM_WORLD,
    [np.array([0, 0]), np.array([1, 1])],
    [nx, ny],
    mesh.CellType.triangle,
)

In [4]:
function_space = fem.functionspace(domain, ("Lagrange", 1))
trial_function = ufl.TrialFunction(function_space)
test_function = ufl.TestFunction(function_space)

mass_matrix_term = ufl.inner(trial_function, test_function) * ufl.dx
stiffness_matrix_term = ufl.inner(ufl.grad(trial_function), ufl.grad(test_function)) * ufl.dx
spde_matrix_term = delta * mass_matrix_term + gamma * stiffness_matrix_term

mass_matrix = petsc.assemble_matrix(fem.form(mass_matrix_term))
stiffness_matrix = petsc.assemble_matrix(fem.form(stiffness_matrix_term))
mass_matrix.assemble()
stiffness_matrix.assemble()

In [5]:
dense_mass_matrix_automatic = mass_matrix.convert("dense")
dense_mass_array_automatic = dense_mass_matrix_automatic.getDenseArray()

In [6]:
vertex_coordinates = domain.geometry.x
cell_vertex_map = domain.geometry.dofmap
cell_dof_map = function_space.dofmap
index_map = cell_dof_map.index_map
local_cell_range = domain.topology.index_map(domain.topology.dim).local_range
num_local_cells = local_cell_range[1] - local_cell_range[0]
cell_matrices = []
global_dofs = []

mass_matrix_compiled, *_ = jit.ffcx_jit(
    domain.comm,
    mass_matrix_term,
    form_compiler_options={"scalar_type": PETSc.ScalarType},
)
compiled_kernel = getattr(
    mass_matrix_compiled.form_integrals[0],
    f"tabulate_tensor_{np.dtype(PETSc.ScalarType).name}",
)

ffi = cffi.FFI()

In [7]:
num_global_dofs = index_map.size_global
manual_mass_matrix = PETSc.Mat().createAIJ([num_global_dofs, num_global_dofs], 
                              comm=domain.comm)
manual_mass_matrix.setUp()

In [13]:
for cell_index in range(num_local_cells):
    local_cell_dofs = cell_dof_map.cell_dofs(cell_index)
    global_cell_dofs = index_map.local_to_global(local_cell_dofs).astype(PETSc.IntType)
    num_cell_dofs = local_cell_dofs.size
    cell_vertices = cell_vertex_map[cell_index]
    cell_vertex_coordinates = vertex_coordinates[cell_vertices]
    cell_matrix = np.zeros((num_cell_dofs, num_cell_dofs), dtype=PETSc.ScalarType)
    compiled_kernel(
        ffi.from_buffer(cell_matrix),  # A - output matrix
        ffi.NULL,  # w - coefficient values (none for this form)
        ffi.NULL,  # c - constants (none for this form)
        ffi.from_buffer(cell_vertex_coordinates),  # coordinate_dofs
        ffi.NULL,  # entity_local_index (cell integral)
        ffi.NULL,  # cell_orientation
    )
    global_dofs.append(global_cell_dofs)
    cell_matrices.append(np.linalg.cholesky(cell_matrix))
    manual_mass_matrix.setValues(
        global_cell_dofs, global_cell_dofs, cell_matrix, addv=PETSc.InsertMode.ADD_VALUES
    )

In [10]:
dense_mass_matrix_manual = mass_matrix.convert("dense")
dense_mass_array_manual = dense_mass_matrix_manual.getDenseArray()